In [ ]:
!pip install tensorflow_text

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [3]:
df = pd.read_csv("procData.csv")
df.head()

,title,files,description,category,fileAmount,more100Files,fileSize
0,[valoon] All 1982 1.Winter Anime Openings,[valoon] All 1982 1. Winter Anime Openings.mp4,\n Fresh start into 2022 on...,Anime-Anime Music Video,1,0,18.165656
1,[MTBB] GOTCHA! (BD 1080p),[MTBB] GOTCHA! [BD 1080p] [3071BC7C].mkv,\n A Blu-ray finally came o...,Anime-Anime Music Video,1,0,19.964831
2,animetronic - Sol Bianca Clip 'Club Cut' MV (D...,animetronic - Sol Bianca Clip 'Club Cut'.mkv,\n https://myanimelist.net/...,Anime-Anime Music Video,1,0,19.619635
3,[valoon] All 1992 1.Winter Anime Openings,[valoon] All 1992 1. Winter Anime Openings.mp4,"\n Finally, this is getting...",Anime-Anime Music Video,1,0,18.400905
4,[DB-MV]_NHK_Minna_no_Uta_-_Egao,[DB-MV]_NHK_Minna_no_Uta_-_Egao_[2868D747].avi,\n ```\nA music video direc...,Anime-Anime Music Video,1,0,18.021827


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70294 entries, 0 to 70293
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         70294 non-null  object 
 1   files         70294 non-null  object 
 2   description   70294 non-null  object 
 3   category      70294 non-null  object 
 4   fileAmount    70294 non-null  int64  
 5   more100Files  70294 non-null  int64  
 6   fileSize      70294 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 3.8+ MB


In [5]:
train, val, test = np.split(df.sample(frac=1), [int(0.8 * len(df)), int(0.9 * len(df))])

In [6]:
print(len(train), len(val), len(test), len(df))

56235 7029 7030 70294


In [7]:
def df_to_dataset(dataframe, shuffle=True, batch_size=100):
  df = dataframe.copy()
  labels = df.pop('category')
  df = {key: value[:,tf.newaxis] for key, value in df.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [8]:
train_data = df_to_dataset(train)
test_data = df_to_dataset(test)
validation_data = df_to_dataset(val)

<ipython-input-7-e90f28cd74b0>:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in df.items()}


In [9]:
train['category'].value_counts()

Anime-English-translated              19938
Anime-Non-English-translated          15660
Literature-Raw                         5475
Live Action-Raw                        3938
Anime-Raw                              3897
Literature-English-translated          3404
Audio-Lossless                         1498
Audio-Lossy                             905
Live Action-English-translated          812
Live Action-Non-English-translated      201
Software-Games                          147
Live Action-Idol/Promotional Video      147
Pictures-Graphics                        71
Anime-Anime Music Video                  54
Literature-Non-English-translated        52
Software-Applications                    24
Pictures-Photos                          12
Name: category, dtype: int64

In [10]:
catVals = list(set(train['category'].values))
table = tf.lookup.StaticHashTable(
    initializer = tf.lookup.KeyValueTensorInitializer(
        keys = tf.constant(catVals),
        values = list(range(len(catVals)))
    ),
    default_value = -1,
    name = "target_encoding"
)

In [11]:
list(set(train['category'].values))

['Software-Games',
 'Pictures-Photos',
 'Anime-Non-English-translated',
 'Live Action-English-translated',
 'Literature-Non-English-translated',
 'Software-Applications',
 'Anime-English-translated',
 'Literature-English-translated',
 'Live Action-Non-English-translated',
 'Live Action-Raw',
 'Pictures-Graphics',
 'Anime-Anime Music Video',
 'Audio-Lossy',
 'Audio-Lossless',
 'Anime-Raw',
 'Literature-Raw',
 'Live Action-Idol/Promotional Video']

In [12]:
@tf.function
def target(x):
  return table.lookup(x)

In [13]:
def fetch(features, labels):
  return features, tf.one_hot(target(labels), len(catVals))

In [14]:
train_data_f = train_data.map(fetch)
test_data_f = test_data.map(fetch)
validation_data_f = validation_data.map(fetch)

In [15]:
len(next(iter(train_data_f)))

2

In [16]:
embedding = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
hub_layer = hub.KerasLayer(embedding, output_shape = 512, input_shape = [], dtype=tf.string, trainable=True)

In [17]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a layer that turns strings into integer indices.
  if dtype == 'string':
    index = tf.keras.layers.StringLookup(max_tokens=max_tokens)
  # Otherwise, create a layer that turns integer values into integer indices.
  else:
    index = tf.keras.layers.IntegerLookup(max_tokens=max_tokens)

  # Prepare a `tf.data.Dataset` that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Encode the integer indices.
  encoder = tf.keras.layers.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply multi-hot encoding to the indices. The lambda function captures the
  # layer, so you can use them, or include them in the Keras Functional model later.
  return lambda feature: encoder(index(feature))

In [18]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = tf.keras.layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [19]:
all_inputs = []
encoded_features = []

# Numerical features.
for header in ['fileAmount', 'fileSize']:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_data_f)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [20]:
for header in ['more100Files']:
  num_cat_col = tf.keras.Input(shape=(1,), name=header)
  categ_layer = get_category_encoding_layer(header, train_data_f, "int64")
  encoded_categ_col = categ_layer(num_cat_col)
  all_inputs.append(num_cat_col)
  encoded_features.append(encoded_categ_col)

In [21]:
for header in ['title', 'description', 'files']:
  text_col = tf.keras.Input(shape=(), name=header, dtype='string')
  encoded_text_col = hub_layer(text_col)
  all_inputs.append(text_col)
  encoded_features.append(encoded_text_col)

In [22]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = all_features
for i in range(3):
  x = tf.keras.layers.Dense(16, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.4)(x)
output = tf.keras.layers.Dense(len(catVals), activation='softmax')(x)

In [23]:
model = tf.keras.Model(all_inputs, output)

In [24]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [25]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 more100Files (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 fileAmount (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 fileSize (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 integer_lookup (IntegerLookup)  (None, 1)           0           ['more100Files[0][0]']           
                                                                                              

In [26]:
history = model.fit(train_data_f, epochs=5, validation_data=validation_data_f)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/keras/backend.py:5534: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


563/563 [==============================] - 584s 871ms/step - loss: 1.7855 - accuracy: 0.4280 - val_loss: 1.0163 - val_accuracy: 0.5824
Epoch 2/5
563/563 [==============================] - 236s 420ms/step - loss: 1.1275 - accuracy: 0.5353 - val_loss: 0.8423 - val_accuracy: 0.6216
Epoch 3/5
563/563 [==============================] - 222s 394ms/step - loss: 1.0406 - accuracy: 0.5610 - val_loss: 0.8419 - val_accuracy: 0.6201
Epoch 4/5
563/563 [==============================] - 212s 377ms/step - loss: 1.0788 - accuracy: 0.5581 - val_loss: 0.9209 - val_accuracy: 0.5978
Epoch 5/5
563/563 [==============================] - 212s 376ms/step - loss: 1.0544 - accuracy: 0.5595 - val_loss: 0.8886 - val_accuracy: 0.6009


In [27]:
model.evaluate(test_data_f)

71/71 [==============================] - 10s 136ms/step - loss: 0.8989 - accuracy: 0.5989


[0.8989464044570923, 0.5988619923591614]

In [28]:
model.evaluate(validation_data_f)

71/71 [==============================] - 9s 126ms/step - loss: 0.8886 - accuracy: 0.6009


[0.8886199593544006, 0.6009389758110046]

In [29]:
model.save("trainedModel")

In [36]:
test_x = test_data_f.unbatch().map(lambda x, y: x)
test_y = np.concatenate([y for x, y in iter(test_data_f)], axis=0)

In [37]:
test_predicted = model.predict(test_x)

7030/7030 [==============================] - 214s 30ms/step


In [38]:
type(next(iter(test_data_f)))

tuple

In [39]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [40]:
print(classification_report(test_y.argmax(1), test_predicted.argmax(1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        21
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00      1937
           3       0.00      0.00      0.00       118
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00         1
           6       0.37      0.64      0.47      2464
           7       0.18      0.15      0.16       423
           8       0.00      0.00      0.00        27
           9       0.12      0.14      0.13       490
          10       0.00      0.00      0.00        12
          11       0.00      0.00      0.00         8
          12       0.00      0.00      0.00       113
          13       0.00      0.00      0.00       178
          14       0.10      0.25      0.15       496
          15       0.18      0.17      0.17       719
          16       0.00      0.00      0.00        15

    accuracy              

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
print(tf.math.confusion_matrix(np.array(test_y).argmax(1), test_predicted.argmax(1)))

tf.Tensor(
[[   0    0    0    0    0    0    9    1    0    3    0    0    0    0
     6    2    0]
 [   0    0    0    0    0    0    1    0    0    0    0    0    0    0
     1    0    0]
 [   0    0    0    0    0    0 1229  113    0  143    0    0    0    0
   279  173    0]
 [   0    0    0    0    0    0   65    8    0    7    0    0    0    0
    29    9    0]
 [   0    0    0    0    0    0    4    1    0    0    0    0    0    0
     1    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    1    0]
 [   0    0    0    0    0    0 1577   85    0  178    0    0    0    0
   422  202    0]
 [   0    0    0    0    0    0  239   65    0   24    0    0    0    0
    51   44    0]
 [   0    0    0    0    0    0   16    2    0    3    0    0    0    0
     5    1    0]
 [   0    0    0    0    0    0  269   19    0   67    0    0    0    0
    82   53    0]
 [   0    0    0    0    0    0    4    0    0    0    0    0    0    0
     6    2    0]

In [43]:
!zip -r /content/trainedModel.zip /content/trainedModel

  adding: content/trainedModel/ (stored 0%)
  adding: content/trainedModel/saved_model.pb (deflated 70%)
  adding: content/trainedModel/variables/ (stored 0%)
  adding: content/trainedModel/variables/variables.data-00000-of-00001 (deflated 49%)
  adding: content/trainedModel/variables/variables.index (deflated 80%)
  adding: content/trainedModel/keras_metadata.pb (deflated 91%)
  adding: content/trainedModel/fingerprint.pb (stored 0%)
  adding: content/trainedModel/assets/ (stored 0%)


In [44]:
from google.colab import files
files.download('/content/trainedModel.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>